## Import data

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='<Your-API-Key>',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'testproj-donotdelete-pr-fnspbxqr9xv8xk'
object_key = 'bid_analysis.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head(10)



/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,week_end,avg_Wheat,avg_Rice,avg_Corn,min_max_acceptable_price_Wheat_min,min_max_acceptable_price_Wheat_max,min_max_acceptable_price_Rice_min,min_max_acceptable_price_Rice_max,min_max_acceptable_price_Corn_min,min_max_acceptable_price_Corn_max
0,9/21/24,1079,1372,996,4.24,3.91,2.11,3.57,4.09,2.04
1,9/14/24,1481,938,707,4.21,2.75,4.19,4.19,2.23,2.90
2,9/7/24,669,553,905,3.81,3.89,3.07,2.23,2.24,2.53
3,8/31/24,1196,955,509,3.74,4.04,4.47,3.06,2.33,2.18
4,8/24/24,820,876,1402,2.34,3.93,3.88,2.33,4.48,2.36
5,8/17/24,668,950,859,3.57,2.38,3.03,3.72,4.15,2.22
6,8/10/24,1247,893,859,2.79,2.35,2.69,2.21,3.38,3.50
7,8/3/24,527,1368,1367,2.51,3.52,2.47,2.98,3.62,3.96
8,7/27/24,1153,1108,572,4.47,4.33,2.33,4.15,3.42,2.91
9,7/20/24,1288,574,517,2.03,2.54,4.31,4.37,2.99,2.72


## Import Libs

When the note book runs the first time install the two libs in the next cell

In [2]:
# !pip install transformers
# ! pip install "tsfm_public[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.8"

In [4]:
import pandas as pd
from tsfm_public import TimeSeriesForecastingPipeline, TinyTimeMixerForPrediction, TimeSeriesPreprocessor
from tsfm_public.toolkit.visualization import plot_predictions

INFO:p-623:t-140290953282560:config.py:<module>:PyTorch version 2.0.1 available.
INFO:p-623:t-140290953282560:config.py:<module>:TensorFlow version 2.12.0 available.
INFO:p-623:t-140290953282560:config.py:<module>:JAX version 0.4.7 available.


In [10]:
## Data Preproc for the Time Series Model 

In [5]:
context_length = 50

data = data.ffill()
data = data.iloc[::-1]


data = data.iloc[-context_length:,]

print(data.shape)
data.head()

(50, 10)


,week_end,avg_Wheat,avg_Rice,avg_Corn,min_max_acceptable_price_Wheat_min,min_max_acceptable_price_Wheat_max,min_max_acceptable_price_Rice_min,min_max_acceptable_price_Rice_max,min_max_acceptable_price_Corn_min,min_max_acceptable_price_Corn_max
49,10/14/23,945,706,1342,4.09,2.64,4.28,4.29,2.17,2.97
48,10/21/23,1193,1135,650,4.37,3.12,4.37,2.42,2.53,2.48
47,10/28/23,1338,1035,928,3.43,2.59,2.75,2.99,4.13,2.09
46,11/4/23,775,1077,563,3.88,3.67,2.21,2.86,3.35,4.43
45,11/11/23,1416,1176,771,3.60,2.97,2.93,2.95,2.47,2.11


In [6]:
data['week_end'] = pd.to_datetime(data['week_end'])
data.dtypes

/tmp/wsuser/ipykernel_623/2430755692.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['week_end'] = pd.to_datetime(data['week_end'])


week_end                              datetime64[ns]
avg_Wheat                                      int64
avg_Rice                                       int64
avg_Corn                                       int64
min_max_acceptable_price_Wheat_min           float64
min_max_acceptable_price_Wheat_max           float64
min_max_acceptable_price_Rice_min            float64
min_max_acceptable_price_Rice_max            float64
min_max_acceptable_price_Corn_min            float64
min_max_acceptable_price_Corn_max            float64
dtype: object

In [7]:
zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-v1", num_input_channels=1
)
model_parameters = sum(p.numel() for p in zeroshot_model.parameters() if p.requires_grad)
print("TTM Model parameters:", model_parameters)

TTM Model parameters: 805280


In [8]:

# Forecasting parameters
context_length = 80
forecast_length = 4
fewshot_fraction = 0.05

timestamp_column = "week_end"
id_columns = []
target_columns = ["avg_Wheat"]
split_config = {"train": 0.7, "test": 0.2}


column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": id_columns,
    "target_columns": target_columns,
    "control_columns": [],
}

tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=context_length,
    prediction_length=forecast_length,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

from tsfm_public.toolkit.time_series_preprocessor import get_datasets, prepare_data_splits


train_df, valid_df, test_df = prepare_data_splits(data, context_length=context_length, split_config=split_config)

# this call also trains the time series preprocessor (tsp)
train_dataset, valid_dataset, test_dataset = get_datasets(
    tsp, data, split_config, fewshot_fraction=fewshot_fraction, fewshot_location="first"
)

TTM_MODEL_REVISION = "main"
zeroshot_model = TinyTimeMixerForPrediction.from_pretrained("ibm/TTM", revision=TTM_MODEL_REVISION)


from tsfm_public.toolkit.time_series_forecasting_pipeline import TimeSeriesForecastingPipeline

# define a forecasting pipeline
forecast_pipeline = TimeSeriesForecastingPipeline(
    model=zeroshot_model,
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    target_columns=target_columns,
    feature_extractor=tsp,
    explode_forecasts=False,
    inverse_scale_outputs=True,
)

# forecast using the pipeline on the scaled test data
crop_wise_forecast = {}
for crop in target_columns:
    forecasts = forecast_pipeline(tsp.preprocess(test_df))[crop+'_prediction'][0][:4]
    crop_wise_forecast[crop] = forecasts


In [9]:
crop_wise_forecast

{'avg_Wheat': array([1055.01403327, 1043.73067337, 1040.08550809, 1043.84541442])}

### Result
